In [1]:
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.linalg as la
from random import randint
%matplotlib inline
%precision 7
plt.style.use('ggplot')

In [4]:
def data_generator(n):
    """ takes a number n and generates 5 times n data points with pre-specified 5 clusters."""
    mean = [4, 8, 11]
    cov = [[1, 0.7, 0.5], [0.7, 1, 0.5], [0.5, 0.5, 1]]
    data0 = np.random.multivariate_normal(mean, cov, n)
    data0 = np.hstack((data0, np.ones((data0.shape[0],1))))
    mean1 = [6, 12, 8]
    cov1 = [[1, 0.5, 0.5], [0.5, 1, 0.5], [0.5, 0.5, 1]]
    data1 = np.random.multivariate_normal(mean1, cov1, n)
    data1 = np.hstack((data1, np.ones((data1.shape[0],1)) * 2))
    mean2 = [10, 13, 13]
    cov2 = [[1, 0.5,0.5], [0.5, 1, 0.3], [0.5, 0.3, 1]]
    data2 = np.random.multivariate_normal(mean2, cov2, n)
    data2 = np.hstack((data2, np.ones((data2.shape[0],1)) * 3))
    mean3 = [13, 10, 18.5]
    cov3 = [[1, 0.5,0.5], [0.5, 1, 0.5], [0.5, 0.5, 1]]
    data3 = np.random.multivariate_normal(mean3, cov3, n)
    data3 = np.hstack((data3, np.ones((data3.shape[0],1)) * 4))
    mean4 = [4, 6, 6]
    cov4 = [[1, 0.5,0.5], [0.5, 1, 0.5], [0.5, 0.5, 1]]
    data4 = np.random.multivariate_normal(mean4, cov4, n)
    data4 = np.hstack((data4, np.ones((data4.shape[0],1)) * 5))
    data = np.vstack((data0, data1, data2, data3,data4))
    np.random.shuffle(data)
    #print (data.shape)
    XX = data [:,0:3]
    data [:,3] = data [:,3]-1
    yy = data [:,3]
    return(XX,yy)

In [5]:
def data_generator2(n):
    """ takes a number n and generates 3 times n data points with pre-specified 3 clusters."""
    mean = [20, 20]
    cov = [[1, 0.7], [0.7, 1]]
    data0 = np.random.multivariate_normal(mean, cov, n)
    data0 = np.hstack((data0, np.ones((data0.shape[0],1))))
    mean1 = [10, 10]
    cov1 = [[1, 0.5], [0.5, 1]]
    data1 = np.random.multivariate_normal(mean1, cov1, n)
    data1 = np.hstack((data1, np.ones((data1.shape[0],1)) * 2))
    2
    mean2 = [0, 0]
    cov2 = [[1, 0.5], [0.5, 1]]
    data2 = np.random.multivariate_normal(mean2, cov2, n)
    data2 = np.hstack((data2, np.ones((data2.shape[0],1)) * 3))
    data = np.vstack((data0, data1, data2))
    np.random.shuffle(data)
    #print (data.shape)
    XX = data [:,0:2]
    data [:,2] = data [:,2]-1
    yy = data [:,2]
    return(XX,yy)

In [9]:
def squared_euclidean_norm(u, axis=-1):
    return((u**2).sum(axis))
def euclidean_norm(u, axis=-1):
    return(np.sqrt(squared_euclidean_norm(u, axis)))
def squared_euclidean_dist(u, v, axis=-1):
    """Returns squared Euclidean distance between two vectors."""
    return(squared_euclidean_norm(u-v, axis))
def min_squared_euclidean_dist(uV, v, axis=-1):
    """Returns the minimum of Euclidean distance between a list of vectors and a vector v"""
    minimum_dis_array = squared_euclidean_dist(uV[0],v)
    for tlk in range(0,len(uV)):
        minimum_dis_array = np.minimum(squared_euclidean_dist(uV[tlk],v),minimum_dis_array)
    return(minimum_dis_array)
def euclidean_dist(u, v, axis=-1):
    """Return Euclidean distacne between two vectors."""
    return(np.sqrt(squared_euclidean_dist(u, v, axis)))

In [17]:
def K_Means_basic(k,DATA, initial_centroid,advanced = False, isCython =False):
    updated_centroid = initial_centroid
    if not advanced: ## meaning it is just a basic version, we sample 5 random points as our cluster
        updated_centroid = DATA[np.random.choice(range(DATA.shape[0]),k), :]
    count = 0
    count2 = 0
    n_feature = len(DATA[0])
    assignment = np.ones(len(DATA))
    assignment2 = np.zeros(len(DATA))
    while not np.array_equal(assignment,assignment2) : ##if the labeling of cluster do not change
        changassignment2=assignment.copy()
    for i in range(0,len(DATA)):
        count = count +1
    if not isCython:
        dist_array = euclidean_dist(updated_centroid,DATA[i]) # find the distance array if isCython:
    dist_array = euclidean_dist_cython(updated_centroid,DATA[i])
    closest_distance = min(dist_array)
    closest_index = np.where(dist_array[:]==closest_distance)
    assignment[i] = min(np.asarray(closest_index))[0]
    for jj in range(0,k):
        ##updates the cnetroid
        updated_centroid[jj] = np.mean(DATA[np.where(assignment[:]==jj)][:,0:n_feature], axis=2)
    count2 = count2 +1
    return(updated_centroid,assignment)

In [13]:
def K_Means_plusplus(k,DATA,isCython = False):
    random_point_index = randint(0,len(DATA))
    center = []
    center.append(DATA[random_point_index]) #Sample a point uniformly at random from X
    phi_x_C_vector = squared_euclidean_dist(center[0],DATA)
    p_x = phi_x_C_vector/sum(phi_x_C_vector) #Calculate the weight probability
    cluster2 = []
    while len(center) < k:
        nnn = np.random.multinomial(1,p_x).tolist() # sample a point with the weight probability
        loc1 = nnn.index(max(nnn))
        center.append(DATA[loc1])
        phi_x_C_vector = min_squared_euclidean_dist(center,DATA)
        p_x = phi_x_C_vector/sum(phi_x_C_vector) #updates the weight probability
    return K_Means_basic(k,DATA,np.array(center),True,isCython)

In [ ]:
def Scalable_K_Means(k,l,DATA,isCython=False):
    random_point_index = randint(0,len(DATA))
    center = []
    center.append(DATA[random_point_index]) #Sample a point uniformly at random from X
    
    phi_x_C_vector = squared_euclidean_dist(center[0],DATA)
    p_x = (l*phi_x_C_vector) / sum(phi_x_C_vector) #Calculate the weight probability
    
    for itjj in range(0,int(np.log(sum(phi_x_C_vector)))):
        uniform_p = np.random.uniform(0,1,len(DATA))
        
        for itj in range(0,len(DATA)):
            if uniform_p[itj] < p_x[itj]:
                center.append(DATA[itj]) # sample each point x<- X independently
                
            phi_x_C_vector = min_squared_euclidean_dist(center,DATA) 
            p_x = l*phi_x_C_vector/sum(phi_x_C_vector)
            
    w_x_vector = np.zeros(len(center))
    
    if isCython: #put if statment outside of the for loop
        center_array = np.asarray(center)
        for i in range(0,len(DATA)):
            nn = euclidean_dist_cython(center_array,DATA[i]).tolist()
            loc = nn.index(min(nn))
            w_x_vector[loc] = w_x_vector[loc] + 1
    if not isCython:
        for i in range(0,len(DATA)):
            nn = euclidean_dist(center,DATA[i]).tolist()
            loc = nn.index(min(nn))
            w_x_vector[loc] = w_x_vector[loc] + 1
    
    w_x_vector_prob = w_x_vector/sum(w_x_vector)
    center2 = []
    
    while len(center2) < k:
        nnn = np.random.multinomial(1,w_x_vector_prob).tolist() # sample a point with the weight 
        loc1 = nnn.index(max(nnn))
        center2.append(center[loc1])
        phi_x_C_vector = min_squared_euclidean_dist(center2,center)
        p_x = phi_x_C_vector/sum(phi_x_C_vector)
    #print center2
    return(K_Means_basic(k,DATA,np.asarray(center2),True,isCython))